In [16]:
import pandas as pd
import numpy as np
import time
import re as re
from selenium import webdriver
import translators as ts
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



driver = webdriver.Chrome()
def get_data_lundbeck(url):
    
    """
    This function will collect the grant information provided from the Lundbeck foundation grant database with the url: "https://lundbeckfonden.com/grants-prizes/what-we-have-funded/all-grants"
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    year_l = []
    project_title_l = []
    project_category_l = []
    receiver_name_l = []
    receiver_title_l = []
    receiver_organisation_l = []
    amount_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url (https://lundbeckfonden.com/grants-prizes/what-we-have-funded/all-grants")
    return: Dataframe with text extracted
    """

    year_l = []
    project_title_l = []
    project_category_l = []
    receiver_name_l = []
    receiver_title_l = []
    receiver_organisation_l = []
    amount_l = []
    
    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    time.sleep(5)
    
    #click on the #donations button
    action = ActionChains(driver)
    
    try:
        for page_num in range(1,137): #137
            """we click next page 137 times"""
            #time.sleep(5)
            #ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/div[2]/main/div/div/article/div[2]/div[1]/div/div/div[2]/table/tbody/tr[1]/td[1]")).perform()  #move to place
            for i in range(1,26):
                time.sleep(3)
                action.move_to_element(driver.find_element(By.CSS_SELECTOR, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-title")).perform()  #move to the line of interest
                
                #year
                test_function(year_l, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-field-funding-year")
                #Project Category
                test_function(project_category_l, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-field-category")
                #Receiver Name
                test_function(receiver_name_l, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-field-project-applicant.views-field-field-applicant-title.views-field-field-applicant-institution > div.applicant-name")
                #Receiver Title
                test_function(receiver_title_l, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-field-project-applicant.views-field-field-applicant-title.views-field-field-applicant-institution > div.application-position")
                #Receiver organisation
                test_function(receiver_organisation_l, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-field-project-applicant.views-field-field-applicant-title.views-field-field-applicant-institution > div.applicant-institution")
                #procejt title
                test_function(project_title_l, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-title")
                #Amount
                test_function(amount_l, f"body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > div.view-content > table > tbody > tr:nth-child({i}) > td.views-field.views-field-field-funding-amount-number")

                if i == 25 and page_num == 1:

                    ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, "body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > nav > ul > li.pager__item.pager__item--next > a")).perform()  #move to click button
                    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > nav > ul > li.pager__item.pager__item--next > a")))
                    element.click()
                    print("pagenumber", page_num)
                elif i == 25 and page_num > 1:
                    ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, "body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > nav > ul > li.pager__item.pager__item--next > a > span:nth-child(2)")).perform()  #move to click button
                    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.dialog-off-canvas-main-canvas > main > div > div > article > div.footer > div.field.field-viewsreference > div > div > nav > ul > li.pager__item.pager__item--next > a > span:nth-child(2)")))
                    element.click()
                    print("pagenumber", page_num)

                    

            
    except NoSuchElementException: #avoid programme crashing
        print("Loop ended")
        print(i)
        print("pagenumber", page_num)

 

    df_final = pd.DataFrame(list(zip(year_l, receiver_organisation_l, amount_l, project_title_l, receiver_name_l, receiver_title_l, project_category_l)), columns=["Year",'Reciever','Grant size (DKK)', 'Description', "Receiver Name", "Receiver Proffession", "Project Category"])

    return df_final

def test_function(listname, css_selector_p):
    
    """
    helper function, that will first try to find the right element, 
    and if an error is raied, it will insert an "NA" instead
    """
    
    try:
        text_found = driver.find_element(By.CSS_SELECTOR, css_selector_p).text
        listname.append(text_found)
    except ElementClickInterceptedException:
        text_found = "NA"
        listname.append(location)
    except StaleElementReferenceException:
        text_found = "NA"
        listname.append(location)
    except NoSuchElementException:
        text_found = "NA"
        listname.append(text_found)
        
        return


biglist = get_data_lundbeck("https://lundbeckfonden.com/grants-prizes/what-we-have-funded/all-grants")
pd.set_option('display.max_columns', None)

biglist.to_csv('lundbeck_data.csv', index=True)

pagenumber 1
pagenumber 2
pagenumber 3
pagenumber 4
pagenumber 5
pagenumber 6
pagenumber 7
pagenumber 8
pagenumber 9
pagenumber 10
pagenumber 11
pagenumber 12
pagenumber 13
pagenumber 14
pagenumber 15
pagenumber 16
pagenumber 17
pagenumber 18
pagenumber 19
pagenumber 20
pagenumber 21
pagenumber 22
pagenumber 23
pagenumber 24
pagenumber 25
pagenumber 26
pagenumber 27
pagenumber 28
pagenumber 29
pagenumber 30
pagenumber 31
pagenumber 32
pagenumber 33
pagenumber 34
pagenumber 35
pagenumber 36
pagenumber 37
pagenumber 38
pagenumber 39
pagenumber 40
pagenumber 41
pagenumber 42
pagenumber 43
pagenumber 44
pagenumber 45
pagenumber 46
pagenumber 47
pagenumber 48
pagenumber 49
pagenumber 50
pagenumber 51
pagenumber 52
pagenumber 53
pagenumber 54
pagenumber 55
pagenumber 56
pagenumber 57
pagenumber 58
pagenumber 59
pagenumber 60
pagenumber 61
pagenumber 62
pagenumber 63
pagenumber 64
pagenumber 65
pagenumber 66
pagenumber 67
pagenumber 68
pagenumber 69
pagenumber 70
pagenumber 71
pagenumber 72
p